In [1]:
import numpy as np
all_selected_features = np.load('all_selected_features.npy', encoding = 'latin1')

In [2]:
tags = np.load('tags.npy', encoding = 'latin1')

In [3]:
all_selected_features.shape

(1015, 612)

In [12]:
def num(i,tags):
    c=0
    for j in tags:
        if j==i:
            c+=1
    return c
for i in range (1,22):
    print(i,num(i,tags))

1 1
2 24
3 48
4 12
5 24
6 12
7 24
8 8
9 16
10 12
11 24
12 12
13 42
14 216
15 6
16 37
17 7
18 2
19 72
20 6
21 6


In [4]:
from sklearn.utils import shuffle
all_features, all_MOS = shuffle(all_selected_features[:,1:], all_selected_features[:,0]/5, random_state=1 )

In [5]:
from sklearn import tree
regr = tree.DecisionTreeRegressor(random_state=1)
#
from sklearn.ensemble import AdaBoostRegressor
regr = AdaBoostRegressor(tree.DecisionTreeRegressor(max_depth=4,random_state=1),
                         n_estimators=50, random_state=1)
from sklearn.svm import SVR
regr = SVR(kernel='linear', C=1.0, epsilon=0.2)
regr = AdaBoostRegressor(SVR(kernel='linear', C=1.0, epsilon=0.2),
                         n_estimators=50, random_state=1)

from sklearn.ensemble import ExtraTreesRegressor
regr = ExtraTreesRegressor(n_estimators=100,random_state=1)

from sklearn.ensemble import RandomForestRegressor
regr = RandomForestRegressor(n_estimators=50,random_state=1)

from sklearn import ensemble
params = {'n_estimators': 500, 'max_depth': 4, 'min_samples_split': 2,
      'learning_rate': 0.05, 'loss': 'ls'}
regr = ensemble.GradientBoostingRegressor(**params) 


from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel,ExpSineSquared
kernel = DotProduct() + WhiteKernel()
reg = GaussianProcessRegressor(
        kernel=kernel,
        n_restarts_optimizer=5,
        alpha=1e-5,
        random_state=0)

In [6]:
def MSE(seq1, seq2, length):
    sum_square_error = 0
    for i in range(length):
        sum_square_error += (seq1[i]-seq2[i])**2
        
    mean_square_error =sum_square_error/length
    return mean_square_error

import math
def RMSE(seq1, seq2, length):
    return math.sqrt(MSE(seq1, seq2, length))

def MAE(seq1, seq2, length):
    sum_abs_error = 0
    for i in range(length):
        sum_abs_error += abs(seq1[i]-seq2[i])
        
    mean_abs_error = sum_abs_error/length
    return mean_abs_error



from sklearn.model_selection import KFold
import scipy as scy
sum_spearmanr_train = sum_pearsonr_train = sum_kendalltau_train =  sum_MSE_train = sum_RMSE_train = sum_MAE_train = 0
sum_spearmanr_test  = sum_pearsonr_test  = sum_kendalltau_test  =  sum_MSE_test  = sum_RMSE_test  = sum_MAE_test  = 0
num_folds = 10
k=0
kf = KFold(n_splits=num_folds,random_state=1)
for train, test in kf.split(all_selected_features):
    
    all_features_train, all_MOS_train = [all_selected_features[train,1:], all_selected_features[train,0]/5.0]
    all_features_test,  all_MOS_test  = [all_selected_features[test,1:],  all_selected_features[test,0]/5.0 ]
    
    train_num = all_features_train.shape[0]
    test_num  = all_features_test.shape[0]
    
    list_features_train = []
    list_features_test  = []
    
    list_MOS_train = []
    list_MOS_test  = []
    
    for i in range(train_num):
        list_features_train.append(all_features_train[i,:])
        list_MOS_train.append(all_MOS_train[i])
    
    for i in range(test_num):
        list_features_test.append(all_features_test[i,:])
        list_MOS_test.append(all_MOS_test[i])
        
    # redifine regr
    ## NOTE: use any of the regressors in the above cell and replace with the below regressor.
    kernel = DotProduct() + WhiteKernel()
    reg = GaussianProcessRegressor(
            kernel=kernel,
            n_restarts_optimizer=5,
            alpha=1e-5,
            random_state=0) 

    regr = regr.fit(np.array(list_features_train), np.array(list_MOS_train))
    
    ##---------------------------------------
    k+=1
    print("------------- train ",k,"/",num_folds,"-------------")
    MOS_pred = regr.predict(np.array(list_features_train))
    
    spearmanr_train=scy.stats.spearmanr(list_MOS_train,MOS_pred)
    #print("Spearman Correlation result of train")
    print(spearmanr_train)
    
    pearsonr_train=scy.stats.pearsonr(list_MOS_train,MOS_pred)
    print("Pearson Correlation result of train")
    print(pearsonr_train)
    
    kendalltau_train=scy.stats.kendalltau(list_MOS_train,MOS_pred)
    #print("Kendall's  Correlation result of train")
    print(kendalltau_train)
    
    MSE_train=MSE(list_MOS_train,MOS_pred,len(list_MOS_train))
    print("MSE of train")
    print(MSE_train)
    
    RMSE_train=RMSE(list_MOS_train,MOS_pred,len(list_MOS_train))
    print("RMSE of train")
    print(RMSE_train)
    
    MAE_train=MAE(list_MOS_train,MOS_pred,len(list_MOS_train))
    print("MAE of train")
    print(MAE_train)
    
    
    ##---------------------------------------
    print("------------- test -------------")
    
    MOS_pred = regr.predict(np.array(list_features_test))
    
    spearmanr_test = scy.stats.spearmanr(list_MOS_test,MOS_pred)
    #print("Spearman Correlation result of test")
    print(spearmanr_test)
    
    pearsonr_test=scy.stats.pearsonr(list_MOS_test,MOS_pred)
    print("Pearson Correlation result of test")
    print(pearsonr_test)
    
    kendalltau_test=scy.stats.kendalltau(list_MOS_test,MOS_pred)
    #print("Kendall's  Correlation result of test")
    print(kendalltau_test)
    
    MSE_test=MSE(list_MOS_test,MOS_pred,len(list_MOS_test))
    print("MSE of test")
    print(MSE_test)
    
    RMSE_test=RMSE(list_MOS_test,MOS_pred,len(list_MOS_test))
    print("RMSE of test")
    print(RMSE_test)
    
    MAE_test=MAE(list_MOS_test,MOS_pred,len(list_MOS_test))
    print("MAE of test")
    print(MAE_test)
    
    

    sum_spearmanr_train += spearmanr_train[0]
    sum_pearsonr_train  += pearsonr_train[0]
    sum_kendalltau_train+= kendalltau_train[0]
    sum_MSE_train += MSE_train 
    sum_RMSE_train += RMSE_train 
    sum_MAE_train += MAE_train 
    
    sum_spearmanr_test += spearmanr_test[0]
    sum_pearsonr_test  += pearsonr_test[0]
    sum_kendalltau_test+= kendalltau_test[0]
    sum_MSE_test += MSE_test 
    sum_RMSE_test += RMSE_test 
    sum_MAE_test += MAE_test 

------------- train  1 / 10 -------------
SpearmanrResult(correlation=0.999107958871391, pvalue=0.0)
Pearson Correlation result of train
(0.9993448047547754, 0.0)
KendalltauResult(correlation=0.9793002534239497, pvalue=0.0)
MSE of train
2.5491705600492684e-05
RMSE of train
0.00504893113445734
MAE of train
0.0039840798900285645
------------- test -------------
SpearmanrResult(correlation=0.7351839302809071, pvalue=1.397661896922547e-18)
Pearson Correlation result of test
(0.749032231769928, 1.3927429822919625e-19)
KendalltauResult(correlation=0.5472466922710638, pvalue=4.534883587436388e-16)
MSE of test
0.007448780926868814
RMSE of test
0.0863063203182062
MAE of test
0.06936301920864327
------------- train  2 / 10 -------------
SpearmanrResult(correlation=0.9987386942804688, pvalue=0.0)
Pearson Correlation result of train
(0.9990670218037009, 0.0)
KendalltauResult(correlation=0.9746540355239748, pvalue=0.0)
MSE of train
3.5992857338485636e-05
RMSE of train
0.0059994047486801256
MAE of t

In [7]:

sum_spearmanr_train = sum_spearmanr_train/float(num_folds)
sum_pearsonr_train  = sum_pearsonr_train/float(num_folds)  
sum_kendalltau_train= sum_kendalltau_train/float(num_folds)
sum_MSE_train = sum_MSE_train /float(num_folds)
sum_RMSE_train = sum_RMSE_train /float(num_folds)
sum_MAE_train = sum_MAE_train /float(num_folds)

sum_spearmanr_test = sum_spearmanr_test/float(num_folds)
sum_pearsonr_test  = sum_pearsonr_test/float(num_folds)  
sum_kendalltau_test= sum_kendalltau_test/float(num_folds)
sum_MSE_test = sum_MSE_test /float(num_folds)
sum_RMSE_test = sum_RMSE_test /float(num_folds)
sum_MAE_test = sum_MAE_test /float(num_folds)


In [8]:
print("------------- train -------------")
print("Spearman Correlation result of train")
print(sum_spearmanr_train)

print("Pearson Correlation result of train")
print(sum_pearsonr_train)

print("Kendall's  Correlation result of train")
print(sum_kendalltau_train)

print("MSE of train")
print(sum_MSE_train)

print("RMSE of train")
print(sum_RMSE_train)

print("MAE of train")
print(sum_MAE_train)

#####################################
print("------------- test -------------")
print("Spearman Correlation result of test")
print(sum_spearmanr_test)

print("Pearson Correlation result of test")
print(sum_pearsonr_test)

print("Kendall's  Correlation result of test")
print(sum_kendalltau_test)

print("MSE of test")
print(sum_MSE_test)

print("RMSE of test")
print(sum_RMSE_test)

print("MAE of test")
print(sum_MAE_test)



# Plot
import matplotlib.pyplot as plt
MOS_pred = regr.predict(np.array(list_features_train))

plt.scatter(list_MOS_train,MOS_pred , alpha=0.5)
plt.title('Train data')
plt.xlabel('Ground-truth MOS')
plt.ylabel('Predcited MOS')
plt.savefig('SVR_train',dpi=200)
plt.show()


MOS_pred = regr.predict(np.array(list_features_test))

plt.scatter(list_MOS_test,MOS_pred , alpha=0.5)
plt.title('Test data')
plt.xlabel('Ground-truth MOS')
plt.ylabel('Predcited MOS')
plt.savefig('SVR_test',dpi=200)
plt.show()

------------- train -------------
Spearman Correlation result of train
0.9989194841878473
Pearson Correlation result of train
0.999188298011178
Kendall's  Correlation result of train
0.9768763406188805
MSE of train
3.163612679308904e-05
RMSE of train
0.005601752444572205
MAE of train
0.004434224210629644
------------- test -------------
Spearman Correlation result of test
0.7495728376389053
Pearson Correlation result of test
0.7522071876089749
Kendall's  Correlation result of test
0.5583173420630111
MSE of test
0.006899229062765577
RMSE of test
0.08288139992302801
MAE of test
0.06501877972376018


<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

In [9]:
sum_MSE_test**0.5

0.08306159800272071